In [1]:
from src.data_processing import load_data
import itertools
import string
from src.pun_algorithms import *
from src.ngrams import *
from src.string_similarity import levenshtein
import operator
from src.data_processing import print_progress
from nltk import word_tokenize, pos_tag
from src.data_processing import load_cmu
from src.ipatoarpabet import translate
from string import punctuation
from src.pronunciations import phonetic_distance
import os
from pattern.en import lexeme
from src.pronunciations import get_closest_sounding_words as csw
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
task1, task2, task3, min_pairs, strings, pun_strings = load_data()

In [3]:
with open("data/ngram_searchspace/task1_totals.json") as f:
    search_space = json.load(f)

In [4]:
def classification_accuracy(run):
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for i in range(len(task1)):
        gold_pun = task1[i]['pun']
        if run[i] and gold_pun:
            tp += 1
        if not run[i] and not gold_pun:
            tn += 1
        if run[i] and not gold_pun:
            fp += 1
        if not run[i] and gold_pun:
            fn += 1
    
    results = {}
    results['acc'] = (tp + tn) / len(task1)
    results['recall'] = tp / (tp + fn)
    results['prec'] = tp / (tp + fp)
    results['f1'] = ((2*(results['recall'] * results['prec']))
                      / (results['recall'] + results['prec']))
    
    return results

# No Trigram Baseline

In [5]:
def no_trigram_baseline(index):
    current_context = search_space[index]
    for original_trigram in current_context:
        if current_context[original_trigram]['original_frequency'] == 0:
            return True
    return False

In [6]:
results = [no_trigram_baseline(index) for index in range(len(search_space))]

In [7]:
classification_accuracy(results)

{'acc': 0.7825842696629214,
 'f1': 0.8457552809884415,
 'prec': 0.8570274636510501,
 'recall': 0.8347757671125098}

# No Quadgram Baseline

In [8]:
all_quadgrams_in_task1 = set()
for context in task1:
    words = context['words']
    for i in range(len(words)-3):
        all_quadgrams_in_task1.add(' '.join(words[i:i+4]))

In [9]:
all_quadgrams_in_task1 = list(sorted(all_quadgrams_in_task1))

In [14]:
len(all_quadgrams_in_task1)

17332

In [10]:
groupings = defaultdict(list)
for quadgram in all_quadgrams_in_task1:
    groupings[quadgram.split()[0][:3]].append(quadgram)

In [15]:
len(groupings)

1863

In [11]:
def quad_to_dict(text):
    ret = {}
    for line in text.split('\n'):
        lsplit = line.split()
        try:
            ret[' '.join(lsplit[:4])] = int(lsplit[4])
        except:
            pass
    return ret

In [13]:
quadgram_frequencies ={}
for i, beginning_letters in enumerate(groupings):
    try:
        ngram_output = subprocess.check_output(['zcat', get_gram_file(beginning_letters, 4)]).decode('latin-1')
    except:
        raise
        pass
    quad_dict = quad_to_dict(ngram_output)
    for subquads in groupings[beginning_letters]:
        try:
#             print(subquads)
            quadgram_frequencies[subquads] = quad_dict[subquads]
        except KeyError:
            pass
    print_progress(i, len(groupings))

CalledProcessError: Command '['zcat', '/home/doogy/Data/4grams/c1/upper/e/a/r.gz']' returned non-zero exit status 1

In [115]:
with open("data/quadgram_frequencies.json") as f:
    quadgram_frequencies = json.load(f)

In [ ]:
def no_quadgram_baseline(index):
    context = task1[index]['words']
    for i in range(0, len(context)-3):
        if ' '.join(context[i:i+4]) not in quadgram_frequencies:
            return True
    return False

In [ ]:
results = [no_quadgram_baseline(i) for i in range(len(task1))]

In [ ]:
classification_accuracy(results)